In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

# restore saved variables
%store -r summary_property6_ky_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['property_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['property_six_month'].values

In [4]:
#### CART
depth = [4,5,6]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [60]
depth = [2]
learning_rate = [0.5]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [5]:
cart_summary['FL_score'],  ebm_summary['FL_score']

(0.590497998768473, 0.6526351381069668)

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_six_month'].values

## columns 
cols = KY_X.columns

In [9]:
c_grid = {'C':[0.001, 0.002]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [10]:
stumps_summary

{'best_auc': 0.8593478053522734,
 'best_params': {'C': 0.002},
 'number of coefs': 27,
 'dictionary': {'sex1': 0.25146946387648555,
  'age_at_current_charge19.0': 0.4531706949298059,
  'age_at_current_charge25.0': 0.05100427549922842,
  'age_at_current_charge31.0': 0.025793739252634774,
  'age_at_current_charge34.0': 0.1087007862024617,
  'age_at_current_charge35.0': 0.14542739954541561,
  'age_at_current_charge41.0': 0.26687209200190576,
  'age_at_current_charge61.0': -0.02300104996984095,
  'age_at_current_charge70.0': -0.6957274383612919,
  'p_arrest1': -0.5544870903003422,
  'p_arrest2': 0.7312619697969405,
  'p_arrest3': 0.2601997330464618,
  'p_arrest4': 0.2724440697563598,
  'p_arrest6': 0.11032983380536557,
  'p_arrest7': 0.03857935249878129,
  'p_charges1': -0.579280916871496,
  'p_violence1': -0.033663872143743175,
  'p_felony1': 0.6607861777798802,
  'p_felony2': 0.018878291489691435,
  'p_misdemeanor1': -0.18798061944607797,
  'p_property1': 1.954669824546663,
  'p_weapon1'

### RiskSLIM

In [11]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_six_month'].values

## columns 
cols = KY_X.columns

In [20]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.0008, random_state=816).fit(KY_X, KY_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

20

In [21]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [22]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'property_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=2000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/22/19 @ 01:41 PM | 253 rows in lookup table
10/22/19 @ 01:41 PM | ------------------------------------------------------------
10/22/19 @ 01:41 PM | runnning initialization procedure
10/22/19 @ 01:41 PM | ------------------------------------------------------------
10/22/19 @ 01:41 PM | CPA produced 2 cuts
10/22/19 @ 01:41 PM | running naive rounding on 39 solutions
10/22/19 @ 01:41 PM | best objective value: 0.0345
10/22/19 @ 01:41 PM | rounding produced 5 integer solutions
10/22/19 @ 01:41 PM | best objective value is 0.0383
10/22/19 @ 01:41 PM | running sequential rounding on 39 solutions
10/22/19 @ 01:41 PM | best objective value: 0.0345
10/22/19 @ 01:41 PM | sequential rounding produced 6 integer solutions
10/22/19 @ 01:41 PM | best objective value: 0.0356
10/22/19 @ 01:41 PM | polishing 11 solutions
10/22/19 @ 01:41 PM | best objective value: 0.0356
10/22/19 @ 01:41 PM | polishing produced 5 integer solutions
10/22/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 01:41 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0346.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               2000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.032128968447374506
CPXPARAM_MIP_Tolerances_UpperCutoff              0.034582203622640223


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 01:42 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0346.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 01:42 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0348.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

+----------------------------------------------+------------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(-6 + score))   |                  |           |
| ============================================ | ================ | ========= |
| p_property1                                  |         3 points |   + ..... |
| p_arrest3                                    |         1 points |   + ..... |
| ============================================ | ================ | ========= |
| ADD POINTS FROM ROWS 1 to 2                  |            SCORE |   = ..... |
+----------------------------------------------+------------------+-----------+
setting c0 = 0.0 to ensure that intercept is not penalized
10/22/19 @ 01:43 PM | 253 rows in lookup table
10/22/19 @ 01:43 PM | ------------------------------------------------------------
10/22/19 @ 01:43 PM | runnning initialization procedure
10/22/19 @ 01:43 PM | ------------------------------------------------------------
10/22/19 @ 01:43 PM | CPA produc

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 01:43 PM | adding 251 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0353.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 01:45 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0352.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 01:45 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0349.
Tried aggregator 1 time.
Reduced MIP has 42 rows, 44 columns, and 123 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

In [23]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_train_score'])

(0.6369331676636172, 0.7946593575601077)

In [24]:
#### save results
summary_property6_ky_inter_model = {"CART": cart_summary,
                                   "EBM": ebm_summary,
                                   "Stumps": stumps_summary,
                                   "RiskSLIM": riskslim_summary}
%store summary_property6_ky_inter_model

Stored 'summary_property6_ky_inter_model' (dict)


In [26]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Six Month\\"
results = [["Property",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Six Month KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)